<a target="_blank" href="https://colab.research.google.com/github/ftavella/when_should_school_start/blob/main/school_start_times.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## 📊 When Should School Start?  
### Exploring the Impact of School Start Times on Sleep Through Circadian Rhythms Simulation

This interactive dashboard models how school start times influence your sleep duration throughout the year, using simulations grounded in circadian biology. By adjusting key parameters like city and Daylight Saving Time (DST) observance, you can visualize how environmental light cues and school schedules impact your sleep.

### What You Can Do:
- **Select a City**: Choose any city to see local sunrise/sunset patterns and simulate their effects on student sleep.
- **Toggle DST**: Explore scenarios with or without Daylight Saving Time to understand how it impacts wake-up times and sleep duration.
- **Run Simulations Year-Round**: Observe how sleep patterns change throughout the calendar year and identify optimal start times that align with healthy sleep.

In [ ]:
#@title Setup
# %%capture
!pip install dash
!pip install astral
!pip install pysolar
!pip install numbalsoda
!pip install timezonefinder
!pip uninstall -y circadian
!pip install dash-bootstrap-components
!pip install --upgrade git+https://github.com/ftavella/circadian.git@school_light_schedule

import dash
import pytz
import numpy as np
import numba as nb
import pandas as pd
import pysolar as ps
from numba import cfunc
from astral.sun import sun
import plotly.express as px
import matplotlib.pyplot as plt
from astral import LocationInfo
from scipy.stats import circmean
from geopy.geocoders import Nominatim
from circadian.models import Skeldon23
from numbalsoda import lsoda, lsoda_sig
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta
from timezonefinder import TimezoneFinder
from circadian.lights import LightSchedule
from plotly.colors import sample_colorscale
from dash import dcc, html, Input, Output, State, callback

DEFAULT_MODEL_PARAMETERS = {
    # sleep/wake regulation parameters
    'mu': 17.78,
    'chi': 45.0,
    'H0': 13.0,
    'Delta': 1.0,
    'ca': 1.72,
    'sleep_transition': 100.0,
    'sleep_rate': 20.0,
    # light/circadian parameters
    'tauc': 24.2,
    'f': 0.99669,
    'G': 19.9,
    'p': 0.6,
    'k': 0.55,
    'b': 0.4,
    'gamma': 0.23,
    'alpha_0': 0.16,
    'beta': 0.013,
    'I0': 9500.0,
    'kappa': 24.0 / (2.0 * np.pi),
    # circadian modulation of wakefulness parameters
    'c20': 0.7896,
    'alpha21': -0.3912,
    'alpha22': 0.7583,
    'beta21': -0.4442,
    'beta22': 0.0250,
    'beta23': -0.9647,
    # forced wakeup by light
    'forced_wakeup_light_threshold': 1.0,
    'forced_wakeup_weekday_only': 1.0,
    # light schedule parameters
    'school_start_time': 8.0,
    'school_prep_duration': 1.0,
    'school_duration': 6.0,
    'outdoor_peak_lux': 5000.0,
    'outdoor_light_sunrise_time': 6.0,
    'outdoor_light_sunset_time': 18.0,
    'school_lux': 500.0,
    'school_prep_lux': 200.0,
    'outdoor_baseline_lux': 0.0,
    'outdoor_light_slope': 1.0,
}

DEFAULT_INITIAL_CONDITION = np.array(
    [0.23995682, -1.1547196, 0.50529415, 12.83846474, 0.0]
    )

class ModelParameters:
    def __init__(self, parameter_dict: dict=DEFAULT_MODEL_PARAMETERS):
        for key, value in parameter_dict.items():
            setattr(self, key, value)


def outdoor_light_fn(time_of_day, sunrise_time, sunset_time, peak_lux, baseline_lux, slope):
    rise = np.tanh(slope * (time_of_day - sunrise_time))
    set = np.tanh(slope * (sunset_time - time_of_day))
    amplitude = (peak_lux - baseline_lux) / 2
    return baseline_lux + amplitude * (rise + set)
outdoor_light_fn = np.vectorize(outdoor_light_fn)

def current_light(t, day_duration=24.0,
                  school_start_time=DEFAULT_MODEL_PARAMETERS['school_start_time'],
                  school_prep_duration=DEFAULT_MODEL_PARAMETERS['school_prep_duration'],
                  school_duration=DEFAULT_MODEL_PARAMETERS['school_duration'],
                  outdoor_peak_lux=DEFAULT_MODEL_PARAMETERS['outdoor_peak_lux'],
                  outdoor_light_sunrise_time=DEFAULT_MODEL_PARAMETERS['outdoor_light_sunrise_time'],
                  outdoor_light_sunset_time=DEFAULT_MODEL_PARAMETERS['outdoor_light_sunset_time'],
                  school_lux=DEFAULT_MODEL_PARAMETERS['school_lux'],
                  school_prep_lux=DEFAULT_MODEL_PARAMETERS['school_prep_lux'],
                  outdoor_baseline_lux=DEFAULT_MODEL_PARAMETERS['outdoor_baseline_lux'],
                  outdoor_light_slope=DEFAULT_MODEL_PARAMETERS['outdoor_light_slope'],
                  ):
    is_weekday = (t / day_duration) % 7 < 5
    time_of_day = t % day_duration
    outdoor_light = outdoor_light_fn(time_of_day, outdoor_light_sunrise_time,
                                    outdoor_light_sunset_time, outdoor_peak_lux,
                                    outdoor_baseline_lux, outdoor_light_slope)
    if is_weekday:
        # school schedule
        if time_of_day < school_start_time - school_prep_duration:
            return 0.0
        elif time_of_day < school_start_time:
            return school_prep_lux
        elif time_of_day < school_start_time + school_duration:
            return school_lux
        else:
            return outdoor_light
    else:
        # weekend
        return outdoor_light
current_light = np.vectorize(current_light)

@cfunc(lsoda_sig)
def rhs(t, y, dy, params):

    y_np = nb.carray(y, (5,)) # x, xc, n, H, S
    p_np = nb.carray(params, (36,))

    # Unpack parameters
    (
        # sleep/wake regulation parameters
        mu, chi, H0, Delta, ca, sleep_transition,
        sleep_rate,
        # light/circadian parameters
        tauc, f, G, p, k, b,
        gamma, alpha_0, beta, I0,
        kappa,
        # circadian modulation of wakefulness parameters
        c20, alpha21, alpha22, beta21, beta22, beta23,
        # forced wakeup by light
        forced_wakeup_light_threshold, forced_wakeup_weekday_only,
        # light schedule parameters
        school_start_time, school_prep_duration, school_duration,
        outdoor_peak_lux, outdoor_light_sunrise_time, outdoor_light_sunset_time,
        school_lux, school_prep_lux, outdoor_baseline_lux, outdoor_light_slope,
    ) = p_np

    def _outdoor_light(time_of_day, sunrise_time, sunset_time, peak_lux, baseline_lux, slope):
        rise = np.tanh(slope * (time_of_day - sunrise_time))
        set = np.tanh(slope * (sunset_time - time_of_day))
        amplitude = (peak_lux - baseline_lux) / 2
        return baseline_lux + amplitude * (rise + set)


    def _current_light(t, day_duration=24.0,
                       school_start_time=school_start_time,
                       school_prep_duration=school_prep_duration,
                       school_duration=school_duration,
                       outdoor_peak_lux=outdoor_peak_lux,
                       outdoor_light_sunrise_time=outdoor_light_sunrise_time,
                       outdoor_light_sunset_time=outdoor_light_sunset_time,
                       school_lux=school_lux,
                       school_prep_lux=school_prep_lux,
                       outdoor_baseline_lux=outdoor_baseline_lux,
                       outdoor_light_slope=outdoor_light_slope,
                       ):
        is_weekday = (t / day_duration) % 7 < 5
        time_of_day = t % day_duration
        outdoor_light = _outdoor_light(time_of_day, outdoor_light_sunrise_time,
                                       outdoor_light_sunset_time, outdoor_peak_lux,
                                       outdoor_baseline_lux, outdoor_light_slope)
        if is_weekday:
            # school schedule
            if time_of_day < school_start_time - school_prep_duration:
                return 0.0
            elif time_of_day < school_start_time:
                return school_prep_lux
            elif time_of_day < school_start_time + school_duration:
                return school_lux
            else:
                return outdoor_light
        else:
            # weekend
            return outdoor_light

    # Unpack state variables
    x, xc, n, H, S = y_np

    light = _current_light(t)
    received_light = np.abs((1.0 - S) * light)

    # circadian model derivative
    alpha = alpha_0 * pow((received_light / I0), p)
    B = G * (1.0 - n) * alpha * (1.0 - b * x) * (1.0 - b * xc)
    gamma_term = gamma * (xc - 4.0 / 3.0 * pow(xc, 3.0))
    tauc_term = pow(24.0 / (f * tauc), 2.0) + k * B

    dxdt = (1.0 / kappa) * (xc + B)
    dxcdt = (1.0 / kappa) * (gamma_term - x * tauc_term)
    dndt = 60.0 * (alpha * (1.0 - n) - beta * n)

    # sleep pressure signal (H(t)) derivative
    dHdt = (1.0 / chi) * (- H + (1.0 - S) * mu)

    # sleep state
    linear_term = c20 + alpha21 * xc + alpha22 * x
    quadratic_term = beta21 * xc * xc + beta22 * xc * x + beta23 * x * x
    C = linear_term + quadratic_term
    H_plus = H0 + 0.5 * Delta + ca * C
    H_minus = H0 - 0.5 * Delta + ca * C
    threshold = H_plus * (1.0 - S) + H_minus * S
    S_target = 1.0 / (1.0 + np.exp(- sleep_transition * (H - threshold)))
    # forced wakeup by light
    if light > forced_wakeup_light_threshold:
        if forced_wakeup_weekday_only > 0.0:
            is_weekday = (t / 24.0) % 7 < 5
        else:
            is_weekday = True
        if is_weekday:
            S_target = 0.0

    dSdt = sleep_rate * (S_target - S)

    # Update dy in-place
    dy[0] = dxdt
    dy[1] = dxcdt
    dy[2] = dndt
    dy[3] = dHdt
    dy[4] = dSdt

def circadian_modulation_of_sleep(x, xc, mp):
    linear_term = mp.c20 + mp.alpha21 * xc + mp.alpha22 * x
    quadratic_term = mp.beta21 * xc * xc + mp.beta22 * xc * x + mp.beta23 * x * x
    C = linear_term + quadratic_term
    return C

def H_thresholds(x, xc, mp):
    C = circadian_modulation_of_sleep(x, xc, mp)
    H_plus = mp.H0 + 0.5 * mp.Delta + mp.ca * C
    H_minus = mp.H0 - 0.5 * mp.Delta + mp.ca * C
    return H_plus, H_minus

def calculate_sleep_onset_and_offset(time, sleep):
    sleep_onset_idx = np.where(np.diff(sleep) == 1)[0]
    sleep_offset_idx = np.where(np.diff(sleep) == -1)[0]
    if sleep_offset_idx[0] < sleep_onset_idx[0]:
        sleep_offset_idx = sleep_offset_idx[1:]
    if sleep_onset_idx[-1] > sleep_offset_idx[-1]:
        sleep_onset_idx = sleep_onset_idx[:-1]
    min_length = min(len(sleep_onset_idx), len(sleep_offset_idx))
    sleep_onset_idx = sleep_onset_idx[:min_length]
    sleep_offset_idx = sleep_offset_idx[:min_length]
    return time[sleep_onset_idx], time[sleep_offset_idx]


def calculate_sleep_duration(time, sleep):
    sleep_onset, sleep_offset = calculate_sleep_onset_and_offset(time, sleep)
    sleep_duration = sleep_offset - sleep_onset
    return sleep_duration


def calculate_average_weekday_sleep_duration(sleep_onset, sleep_offset):
    weekday_sleep_durations = []
    for onset, offset in zip(sleep_onset, sleep_offset):
        day_of_week = int(onset // 24) % 7  # Determine the day of the week (0=Monday, ..., 6=Sunday)
        if day_of_week < 5:  # Only consider weekdays (Monday to Friday)
            weekday_sleep_durations.append(offset - onset)

    if weekday_sleep_durations:
        return np.mean(weekday_sleep_durations)
    else:
        return 0.0  # Return 0 if no weekday sleep durations are found


def get_city_info(city_name):
    """
    Gets the latitude, longitude, and time zone of a given city.

    Args:
        city_name: The name of the city.

    Returns:
        A dictionary containing the latitude, longitude, and time zone of the city,
        or None if the city is not found or an error occurs.
    """
    geolocator = Nominatim(user_agent="app")
    tf = TimezoneFinder()

    try:
        location = geolocator.geocode(city_name)
        if location:
            latitude = location.latitude
            longitude = location.longitude
            timezone = tf.timezone_at(lng=longitude, lat=latitude)

            return {
                "latitude": latitude,
                "longitude": longitude,
                "timezone": timezone,
            }
        else:
            print(f"City '{city_name}' not found.")
            return None
    except Exception as e:
        print(f"Error getting city info: {e}")
        return None


def get_sunrise_sunset(latitude, longitude, timezone, datetime_value):
    """
    Calculates sunrise and sunset times for a given latitude, longitude, and timezone.

    Args:
        latitude: Latitude of the location.
        longitude: Longitude of the location.
        timezone: Timezone string (e.g., "US/Pacific", "Europe/London").

    Returns:
        A tuple containing sunrise and sunset times as hours of the day in local time,
        or None if an error occurs.
    """
    try:
        # Create a LocationInfo object with the specified timezone
        city = LocationInfo("Example City", "Region", timezone, latitude, longitude)

        # Calculate sunrise and sunset for today
        s = sun(city.observer, date=datetime_value)

        # Convert sunrise and sunset to local time
        local_tz = pytz.timezone(timezone)
        sunrise_local = s['sunrise'].astimezone(local_tz)
        sunset_local = s['sunset'].astimezone(local_tz)

        # Extract hour of day
        sunrise_hour = sunrise_local.hour
        sunset_hour = sunset_local.hour

        return sunrise_hour, sunset_hour

    except Exception as e:
        print(f"Error calculating sunrise/sunset: {e}")
        return None, None


def get_direct_sunlight_lux(datetime_value, timezone, latitude, longitude,
                            wm2_to_lux_conversion_factor=120):
    """
    Calculates the direct sunlight illuminance (lux) at a given location and time.

    Args:
        date: The date and time for which to calculate the illuminance.
        timezone: The timezone of the location.
        latitude: The latitude of the location.
        longitude: The longitude of the location.

    Returns:
        The direct sunlight illuminance in lux, or None if an error occurs.
    """
    try:
        timezone_aware_datetime = pytz.timezone(timezone).localize(datetime_value)
        # Get the solar elevation angle for the location and time
        altitude_deg = ps.solar.get_altitude(latitude, longitude, timezone_aware_datetime)
        # Calculate the illuminance in lux
        illuminance = ps.solar.radiation.get_radiation_direct(datetime_value, altitude_deg)
        lux = illuminance * wm2_to_lux_conversion_factor  # Convert W/m^2 to lux
        return lux

    except Exception as e:
        print(f"Error calculating direct sunlight lux: {e}")
        return None


def is_dst_season(datetime_value):
    if datetime_value.tzinfo is None:
        raise ValueError("The datetime object must be timezone-aware.")
    return bool(datetime_value.dst())

def get_outdoor_light_parameters(city_name, datetime_value, dst_in_effect):
    location_dict = get_city_info(city_name)
    if location_dict is None:
        return None
    else:
        latitude = location_dict["latitude"]
        longitude = location_dict["longitude"]
        timezone = location_dict["timezone"]

        # Get sunrise and sunset times
        sunrise_hour, sunset_hour = get_sunrise_sunset(latitude, longitude, timezone, datetime_value)
        if sunrise_hour is None or sunset_hour is None:
            return None

        direct_sunlight_lux = get_direct_sunlight_lux(datetime_value, timezone, latitude, longitude) # Adjust for DST?

        localized_datetime = pytz.timezone(timezone).localize(datetime_value)

        if dst_in_effect and is_dst_season(localized_datetime):
            # Adjust for daylight saving time
            sunrise_hour += 1
            sunset_hour += 1

        return {
            "latitude": latitude,
            "longitude": longitude,
            "timezone": timezone,
            "sunrise_hour": sunrise_hour,
            "sunset_hour": sunset_hour,
            "direct_sunlight_lux": direct_sunlight_lux,
        }


def symlog(x, linthresh=1.0):
    return np.sign(x) * np.log1p(np.abs(x) / linthresh)


def simulate_school_start_time(school_start_time: float, lux_params: dict, dt=5e-3, weeks_to_simulate=4,
                               atol=1e-4, rtol=1e-4, initial_condition=DEFAULT_INITIAL_CONDITION):
    outdoor_peak_lux = 5000.0 if lux_params['direct_sunlight_lux'] > 5000.0 else lux_params['direct_sunlight_lux']
    sunrise_hour = lux_params['sunrise_hour']
    sunset_hour = lux_params['sunset_hour']

    time = np.arange(0, 24 * 7 * weeks_to_simulate, dt)
    mp = ModelParameters()
    params = np.array([
        mp.mu, mp.chi, mp.H0, mp.Delta, mp.ca, mp.sleep_transition,
        mp.sleep_rate,
        mp.tauc, mp.f, mp.G, mp.p, mp.k, mp.b,
        mp.gamma, mp.alpha_0, mp.beta, mp.I0,
        mp.kappa,
        # circadian modulation of wakefulness parameters
        mp.c20, mp.alpha21, mp.alpha22, mp.beta21, mp.beta22, mp.beta23,
        # forced wakeup by light
        mp.forced_wakeup_light_threshold, mp.forced_wakeup_weekday_only,
        # light schedule parameters
        school_start_time, mp.school_prep_duration, mp.school_duration,
        outdoor_peak_lux, sunrise_hour, sunset_hour,
        mp.school_lux, mp.school_prep_lux, mp.outdoor_baseline_lux, mp.outdoor_light_slope,
    ])
    function_pointer = rhs.address
    solution = lsoda(function_pointer, initial_condition, time,
                    data=params, atol=atol, rtol=rtol)

    _, _, _, _, sleep = solution[0].T
    sleep = np.where(sleep > 0.5, 1, 0)

    sleep_onset, sleep_offset = calculate_sleep_onset_and_offset(time, sleep)

    return time, solution, sleep_onset, sleep_offset

def format_time_label(start_time):
    """Format the time label as AM/PM."""
    if start_time < 12:
        return f"{int(start_time)} AM"
    elif start_time < 1:
        return "12 PM"
    else:
        return f"{int(start_time - 12)} PM"


app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

# Location and date selection
location_DST_row = dbc.Row([
    dbc.Col([ # Location text
        dbc.Row([
            dbc.Col([
                dbc.Label("Enter a city name"),
            ], width="auto"),
            dbc.Col([
                dbc.Input(id="location", type="text", placeholder="Enter a city name", value="Chicago"),
            ]),
        ], align="center"),
    ], width=3, align="center"),
    dbc.Col([ # DST
        dbc.Checklist(id="dst", options=[{"label": "Daylight Savings Time", "value": 1}], value=[],
                      switch=True),
    ], width=2, align="center"),
    dbc.Col([ # Run simulation button
        dbc.Button("Simulate!", id="run_simulation", color="primary"),
    ], width=3, align="center"),
], className="mt-3")

# Sleep metrics plots
sleep_metrics_plots = dbc.Spinner(color="primary", children=
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=px.line(), id="sunrise_sunset_plot"),
        ], width=4),
        dbc.Col([
            dcc.Graph(figure=px.line(), id="average_sleep_plot"),
        ], width=4),
        dbc.Col([
            dcc.Graph(figure=px.line(), id="minimum_sleep_plot"),
        ], width=4),
    ])
)

app.layout = dbc.Container([
    location_DST_row,
    sleep_metrics_plots,
], fluid=True, style={"padding": "20"})

@app.callback(
    [Output("sunrise_sunset_plot", "figure"),
     Output("average_sleep_plot", "figure"),
     Output("minimum_sleep_plot", "figure")],
    [Input("run_simulation", "n_clicks")],
    [State("location", "value"),
     State("dst", "value")],
     prevent_initial_call=False,
)
def update_light_plots(n_clicks, location, dst, year=2025, school_start_times_range=(6, 13),
                       number_of_school_start_times=10):
    # Gather lux params throughout the year
    lux_params_throughout_year = []
    dst_in_effect = bool(dst)
    error_flag = False
    sunrise_values = []
    sunset_values = []
    for month in range(1, 13):
        # create datetime for each first of the month at noon
        datetime_value = datetime.strptime(f"{year}-{month:02d}-01-12:00:00", "%Y-%m-%d-%H:%M:%S")
        try:
            lux_params = get_outdoor_light_parameters(location, datetime_value, dst_in_effect)
            if lux_params is None:
                error_flag = True
                break
            elif any(value is None for value in lux_params.values()):
                error_flag = True
                break
            lux_params_throughout_year.append(lux_params)
        except:
            lux_params_throughout_year.append(np.nan)
            error_flag = True
            break

    if error_flag:
        print("Error!")
        return px.line(), px.line()

    school_start_times = np.linspace(school_start_times_range[0],
                                     school_start_times_range[1],
                                     number_of_school_start_times)

    average_sleep_duration_result = []

    for start_time in school_start_times:
        # Equilibrate simulation
        equilibration_lux_params = lux_params_throughout_year[0]
        _, equilibration_solution, _, _ = simulate_school_start_time(start_time, equilibration_lux_params,
                                                                     weeks_to_simulate=2)
        # Simulate throughout the year
        current_initial_condition = equilibration_solution[0].T[:, -1]
        for month in range(1, 13):
            lux_params = lux_params_throughout_year[month - 1]
            latitude = lux_params['latitude']
            if latitude < 0:
                # Southern Hemisphere
                # skip summer months
                if month in [12, 1, 2]:
                    continue
            elif latitude > 0:
                # Northern Hemisphere
                # skip winter months
                if month in [6, 7, 8]:
                    continue
           # Simulate
            _, solution, sleep_onset, sleep_offset = simulate_school_start_time(start_time, lux_params,
                                                                                initial_condition=current_initial_condition,
                                                                                weeks_to_simulate=2)
            average_sleep = np.mean(sleep_offset - sleep_onset)
            minimum_sleep = np.min(sleep_offset - sleep_onset)
            result = {
                'start_time': start_time,
                'average_sleep': average_sleep,
                'minimum_sleep': minimum_sleep,
                'month': month
            }
            average_sleep_duration_result.append(result)
            current_initial_condition = solution[0].T[:, -1]

    # Sunrise sunset figure
    sunrise_values = []
    sunset_values = []
    # Define the starting date (January 1st of the year)
    year = 2025
    start_date = datetime(year, 1, 1, 12, 0, 0)  # Noon on January 1st
    # Create a list of datetime objects for each day of the year
    calculate_every_days = 60
    days_to_calculate = np.arange(0, 365, calculate_every_days)
    datetime_array = [start_date + timedelta(days=int(day)) for day in days_to_calculate]
    for datetime_value in datetime_array:
        lux_params = get_outdoor_light_parameters(location, datetime_value, dst_in_effect)
        sunrise_values.append(lux_params['sunrise_hour'])
        sunset_values.append(lux_params['sunset_hour'])

    ss_df = pd.DataFrame({
    'day': [m for m in days_to_calculate] + [m for m in days_to_calculate],
    'hour': sunrise_values + sunset_values,
    'type': ['sunrise'] * len(days_to_calculate) + ['sunset'] * len(days_to_calculate),
    'datetime': datetime_array + datetime_array
    })

    sunrise_sunset_fig = px.line(ss_df, x='datetime', y='hour', color='type',
                                 hover_data={'day': False, 'hour': False, 'datetime': True,
                                             'type': False},
                                 labels={'datetime': 'Date'})
    sunrise_sunset_fig.update_xaxes(
        dtick="M1",
        tickformat="%b %d",
        title_text="Date",
        title_font=dict(size=12),
        tickangle=-45,
    )
    sunrise_sunset_fig.update_yaxes(
        dtick=2,
        tickmode='linear',
        title_text="Hour of the day",
        title_font=dict(size=12),
    )
    # legend
    sunrise_sunset_fig.update_layout(
        legend=dict(
            title="",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1,
        )
    )
    sunrise_sunset_fig.update_traces(mode='lines+markers')

    # Average sleep figure
    discrete_colors = sample_colorscale(px.colors.sequential.Agsunset, [i / 12 for i in range(13)])
    average_sleep_df = pd.DataFrame(average_sleep_duration_result)
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    average_sleep_df['month'] = average_sleep_df['month'].apply(lambda x: month_names[x - 1])
    average_sleep_fig = px.line(average_sleep_df, x='start_time', y='average_sleep', color='month',
                                labels={'start_time': 'School start time',
                                        'average_sleep': 'Average sleep duration (hours)',
                                        'month': 'Month'},
                                color_discrete_sequence=discrete_colors,
                                hover_data={'start_time': True, 'average_sleep': True, 'month': True})
    average_sleep_fig.update_traces(mode='lines+markers')
    xlabels = [format_time_label(start_time) for start_time in school_start_times]
    average_sleep_fig.update_xaxes(
        tickvals=school_start_times,
        ticktext=xlabels,
        title_text="School start time",
        title_font=dict(size=12),
        tickangle=-45,
    )
    average_sleep_fig.update_yaxes(
        dtick=0.25,
        tickmode='linear',
        title_text="Average sleep duration (hours)",
        title_font=dict(size=12),
    )
    average_sleep_fig.update_layout(
        legend=dict(
            title="Month",
            orientation="v",
            yanchor="auto",
            y=1.0,
            xanchor="right",
            x=1.2,
        )
    )

    # Minimum sleep figure
    minimum_sleep_df = pd.DataFrame(average_sleep_duration_result)
    minimum_sleep_df['month'] = minimum_sleep_df['month'].apply(lambda x: month_names[x - 1])
    minimum_sleep_fig = px.line(minimum_sleep_df, x='start_time', y='minimum_sleep', color='month',
                                labels={'start_time': 'School start time',
                                        'minimum_sleep': 'Minimum sleep duration (hours)',
                                        'month': 'Month'},
                                color_discrete_sequence=discrete_colors)
    minimum_sleep_fig.update_traces(mode='lines+markers')
    minimum_sleep_fig.update_xaxes(
        tickvals=school_start_times,
        ticktext=xlabels,
        title_text="School start time",
        title_font=dict(size=12),
        tickangle=-45,
    )
    minimum_sleep_fig.update_yaxes(
        dtick=0.25,
        tickmode='linear',
        title_text="Minimum sleep duration (hours)",
        title_font=dict(size=12),
    )
    minimum_sleep_fig.update_layout(
        legend=dict(
            title="Month",
            orientation="v",
            yanchor="auto",
            y=1.0,
            xanchor="right",
            x=1.2,
        )
    )

    return sunrise_sunset_fig, average_sleep_fig, minimum_sleep_fig

In [ ]:
#@title Run application
app.run(jupyter_width="100%")